# Global Drifter Program (GDP)

As part of this Notebook, we will use the *GDP historical dataset* to highlight the required steps to preprocess and dataset into a format that can be ingest by the *CloudDrift* library.

## Dataformat module

The `dataformat.py` module contains the class `create_ragged_array` to transform a series of archives into an *Awkward Array* where all variables is stored as a *ragged array*. The module also contains `read_from_netcdf` and `read_from_parquet` to initialize the *Awkward Array* directly from an previously preprocessed archive. Right now, it *only* supports local array but we will soon add the possibility of *lazy-loading* array stored in the Cloud.

In [1]:
import sys

In [2]:
sys.path.insert(0, '../')
from clouddrift import dataformat

The main class of this module is *create_ragged_array* and is used to create a single archive that can be saved to a netCDF or Parquet file. The signature of the class is:

In [3]:
dataformat.create_ragged_array?

Init signature:
dataformat.create_ragged_array(
    indices: list,
    preprocess_func: collections.abc.Callable[[int], xarray.core.dataset.Dataset],
    vars_coords: dict,
    vars_meta: list = [],
    vars_data: list = [],
    rowsize_func: collections.abc.Callable[[int], int] = None,
)
Docstring:      <no docstring>
File:           ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/clouddrift/dataformat.py
Type:           type
Subclasses:     


## Dataset-specific functions

Since each dataset is different, we have to create specific functions to preprocess the dataset (`preprocess_func`) and return the metadata and data of a single trajectory. This was inspired by the [Pangeo Forge](https://pangeo-forge.readthedocs.io/en/latest/) project. The class *create_ragged_array* will use those functions to create the single archive of ragged arrays. More precisely, it requires:
- a list of indices (or identification number) that will be concatenate into the ragged array format
- a dictionnary mapping the mandatory coordinates list to the name of those variables in the dataset, e.g.
    coords = {'ids': 'number', 'time': 't', 'longitude': 'lon', 'latitude': 'lat'}
- a list of variable names containing metadata information about the trajectory (size: 1 per trajectory)
- a list of variable names containing the data along the trajectory (size: number of observations per trajectory)
- a preprecessing function with the following signature:
    - `Signature: preprocess_func(index: int) -> xarray.core.dataset.Dataset`, where the index parameter is an identifier of a trajectory, e.g. the identification number of an Argo float) and returns an *xarray Dataset*. 
    
This function can performs all type of operations, such as formatting the date, changing the type of variables, modifying the metadata, etc. We provide preprocessing function for different datasets in the `data/recipes/` folder. The class also needs to *initially* calculate the sum of all observations. By default, this is performed using `lambda i: self.preprocess_func(i).dims['obs']`. To *speed up* this process, in the situation where a lot of preprocessing are performed, it is possible to provide a second function `rowsize_func`, that returns directly the number of observation of a trajectory (`Signature: rowsize_func(index: int) -> int`)

Finally, we import the gdp module which contains a function to download (or update) and preprocess the GDP dataset.

In [4]:
from data import gdp

# Download

The download function will store the raw dataset into the `data/raw/` folder specified in the `gdp.py` module. By default `download_gdp_data()` will download the complete GPD dataset (containing 17,324 files as of May 2022) from the AOML `https` server.

**Note**: this Notebook is very similar to the `data-glad.ipynb` Notebook because very few functions have to be created to transform a new dataset. We hope that this will encourage people to use this dataformat and utilize the CloudDrift library.

In [5]:
gdp.download?

Signature: gdp.download(drifter_ids: list = None, n_random_id: int = None)
Docstring:
Download individual netCDF files from the AOML server

:param drifter_ids [list]: list of drifter to retrieve (Default: all)
:param n_random_id [int]: randomly select n drifter netCDF files
:return drifters_ids [list]: list of retrived drifter
File:      ~/Library/CloudStorage/OneDrive-FloridaStateUniversity/projects/clouddrift/data/gdp.py
Type:      function


It is possible to prodive a list of `drifter_ids` to retrieve a subset and/or specified a integer `n_random_id` to randomly retrieve `n` trajectory. The function returns the list of `drifters_ids` that was downloaded, and can be passed to create the ragged array.

In [6]:
drifter_ids = gdp.download(n_random_id=100)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 757094.58it/s]


Once the data downloaded, it is possible to create the ragged array and either save a netCDF, parquet file, or simply output an Awkward Array that can be used for analysis.

In [7]:
coords = {'ids': 'ids', 'time': 'time', 'lon': 'longitude', 'lat': 'latitude'}
metadata = ['ID', 'rowsize', 'WMO', 'expno', 'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat', 'end_lon', 'drogue_lost_date', 'typedeath', 'typebuoy', 'location_type', 'DeployingShip', 'DeploymentStatus', 'BuoyTypeManufacturer', 'BuoyTypeSensorArray', 'CurrentProgram', 'PurchaserFunding', 'SensorUpgrade', 'Transmissions', 'DeployingCountry', 'DeploymentComments', 'ManufactureYear', 'ManufactureMonth', 'ManufactureSensorType', 'ManufactureVoltage', 'FloatDiameter', 'SubsfcFloatPresence', 'DrogueType', 'DrogueLength', 'DrogueBallast', 'DragAreaAboveDrogue', 'DragAreaOfDrogue', 'DragAreaRatio', 'DrogueCenterDepth', 'DrogueDetectSensor']
data = ['ve', 'vn', 'err_lat', 'err_lon', 'err_ve', 'err_vn', 'gap', 'sst', 'sst1', 'sst2', 'err_sst', 'err_sst1', 'err_sst2', 'flg_sst', 'flg_sst1', 'flg_sst2', 'drogue_status']

ra = dataformat.create_ragged_array(drifter_ids,
                         gdp.preprocess,
                         coords, 
                         metadata, 
                         data,
                         rowsize_func=gdp.rowsize
                        )

Calculating the number of observations: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 65.35it/s]
/Users/pmiron/miniforge3/envs/clouddrift/lib/python3.9/site-packages/xarray/coding/variables.py:140: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
Filling the ragged array: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 35.35it/s]


## Export

In [8]:
ra.to_parquet('../data/process/gdp_v2.00.parquet')
ra.to_netcdf('../data/process/gdp_v2.00.nc')

## Import

In [9]:
ds2 = dataformat.read_from_parquet('../data/process/gdp_v2.00.parquet')

In [10]:
ds2.ID

<Array [6428, 13566, ..., 67772920, 68246720] type='100 * int64[parameters=...'>

## Awkward Array

In [11]:
import awkward._v2 as ak

In [12]:
ds = ra.to_awkward()

In [13]:
ak.nanmean(ds.obs.err_lat, axis=1)

<Array [0.00384, 0.00278, ..., 0.00042, 0.000402] type='100 * ?float64'>

In [14]:
ds.fields

['ID',
 'rowsize',
 'WMO',
 'expno',
 'deploy_date',
 'deploy_lat',
 'deploy_lon',
 'end_date',
 'end_lat',
 'end_lon',
 'drogue_lost_date',
 'typedeath',
 'typebuoy',
 'location_type',
 'DeployingShip',
 'DeploymentStatus',
 'BuoyTypeManufacturer',
 'BuoyTypeSensorArray',
 'CurrentProgram',
 'PurchaserFunding',
 'SensorUpgrade',
 'Transmissions',
 'DeployingCountry',
 'DeploymentComments',
 'ManufactureYear',
 'ManufactureMonth',
 'ManufactureSensorType',
 'ManufactureVoltage',
 'FloatDiameter',
 'SubsfcFloatPresence',
 'DrogueType',
 'DrogueLength',
 'DrogueBallast',
 'DragAreaAboveDrogue',
 'DragAreaOfDrogue',
 'DragAreaRatio',
 'DrogueCenterDepth',
 'DrogueDetectSensor',
 'obs']

In [15]:
ds.obs.fields

['ids',
 'time',
 'lon',
 'lat',
 've',
 'vn',
 'err_lat',
 'err_lon',
 'err_ve',
 'err_vn',
 'gap',
 'sst',
 'sst1',
 'sst2',
 'err_sst',
 'err_sst1',
 'err_sst2',
 'flg_sst',
 'flg_sst1',
 'flg_sst2',
 'drogue_status']

### global attributes

In [16]:
ds.layout.parameters

{'attrs': {'title': 'Global Drifter Program hourly drifting buoy collection',
  'history': 'Version 2.00.  Metadata from dirall.dat and deplog.dat',
  'Conventions': 'CF-1.6',
  'date_created': '2022-05-31T16:41:40.747570',
  'publisher_name': 'GDP Drifter DAC',
  'publisher_email': 'aoml.dftr@noaa.gov',
  'publisher_url': 'https://www.aoml.noaa.gov/phod/gdp',
  'licence': 'MIT License',
  'processing_level': 'Level 2 QC by GDP drifter DAC',
  'metadata_link': 'https://www.aoml.noaa.gov/phod/dac/dirall.html',
  'contributor_name': 'NOAA Global Drifter Program',
  'contributor_role': 'Data Acquisition Center',
  'institution': 'NOAA Atlantic Oceanographic and Meteorological Laboratory',
  'acknowledgement': 'Elipot et al. (2022) to be submitted. Elipot et al. (2016). Global Drifter Program quality-controlled hourly interpolated data from ocean surface drifting buoys, version 2.00. NOAA National Centers for Environmental Information. https://agupubs.onlinelibrary.wiley.com/doi/full/10.10

### variable attributes

In [17]:
ds.ID.layout.parameters

{'attrs': {'long_name': 'Global Drifter Program Buoy ID', 'units': '-'}}

In [18]:
ds.obs.sst.layout.parameters

{'attrs': {'long_name': 'Fitted sea water temperature',
  'units': 'Kelvin',
  'comments': 'Estimated near-surface sea water temperature from drifting buoy measurements. It is the sum of the fitted near-surface non-diurnal sea water temperature and fitted diurnal sea water temperature anomaly. Discrepancies may occur because of rounding.'}}